## LDA utilizando Spacy

In [5]:
# Instalar spacy 
!pip install spacy

In [6]:
import pandas as pd
import numpy as np
import re
import gensim
import spacy
from gensim import corpora
from nltk.corpus import stopwords
from pprint import pprint

In [7]:
# modelo en español de spaCy
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ----------- ---------------------------- 3.7/12.9 MB 19.8 MB/s eta 0:00:01
     --------------------- ------------------ 7.1/12.9 MB 17.4 MB/s eta 0:00:01
     --------------------------------- ----- 11.0/12.9 MB 18.1 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 17.2 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [8]:
# Lista de textos de ejemplo
texts = [
    "La inteligencia artificial está revolucionando la industria.",
    "Los avances en aprendizaje profundo permiten nuevas aplicaciones.",
    "La ética en inteligencia artificial es fundamental para su desarrollo.",
    "Muchos algoritmos de aprendizaje no supervisado son complejos.",
    "La visión por computadora es una rama de la inteligencia artificial."
]

In [9]:
# Convertir a minúsculas y eliminar caracteres especiales
texts = [re.sub(r'\W', ' ', text.lower()) for text in texts]

In [10]:
# Cargar modelo en español
nlp = spacy.load("es_core_news_sm")

# Stopwords en español
stop_words = set(stopwords.words("spanish"))

# Función de preprocesamiento
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and not token.is_punct]
    return tokens

# Aplicar preprocesamiento
data_processed = [preprocess(text) for text in texts]

In [11]:
# Crear diccionario
dictionary = corpora.Dictionary(data_processed)

# Crear corpus (bag of words)
corpus = [dictionary.doc2bow(text) for text in data_processed]

In [12]:
# Configurar y entrenar modelo LDA
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, random_state=42, passes=10)

# Mostrar los tópicos encontrados
pprint(lda_model.print_topics())

[(0,
  '0.114*"inteligencia" + 0.114*"artificial" + 0.113*"ser" + 0.049*"rama" + '
  '0.049*"computadora" + 0.049*"visión" + 0.049*"fundamental" + 0.049*"ética" '
  '+ 0.049*"desarrollo" + 0.048*"industria"'),
 (1,
  '0.099*"aprendizaje" + 0.092*"profundo" + 0.092*"avance" + 0.092*"permitir" '
  '+ 0.092*"aplicación" + 0.092*"nuevo" + 0.033*"algoritmo" + 0.032*"complejo" '
  '+ 0.032*"supervisado" + 0.032*"ser"')]


In [13]:
# Asignación de tópicos a cada texto
for idx, text in enumerate(corpus):
    topic_probabilities = lda_model.get_document_topics(text)
    print(f"\nTexto {idx + 1}: '{texts[idx]}'")
    print("Probabilidades de tópicos:", topic_probabilities)


Texto 1: 'la inteligencia artificial está revolucionando la industria '
Probabilidades de tópicos: [(0, 0.89394575), (1, 0.10605429)]

Texto 2: 'los avances en aprendizaje profundo permiten nuevas aplicaciones '
Probabilidades de tópicos: [(0, 0.07431644), (1, 0.92568356)]

Texto 3: 'la ética en inteligencia artificial es fundamental para su desarrollo '
Probabilidades de tópicos: [(0, 0.9242881), (1, 0.07571187)]

Texto 4: 'muchos algoritmos de aprendizaje no supervisado son complejos '
Probabilidades de tópicos: [(0, 0.887001), (1, 0.11299909)]

Texto 5: 'la visión por computadora es una rama de la inteligencia artificial '
Probabilidades de tópicos: [(0, 0.9243235), (1, 0.07567648)]


### LDA scikit-learn

In [8]:
import numpy as np
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# 1. Conjunto de textos de ejemplo
documents = [
 "La inteligencia artificial está revolucionando la industria.",
    "Los avances en aprendizaje profundo permiten nuevas aplicaciones.",
    "La ética en inteligencia artificial es fundamental para su desarrollo.",
    "Muchos algoritmos de aprendizaje no supervisado son complejos.",
    "La visión por computadora es una rama de la inteligencia artificial."
]

# 2. Preprocesamiento del texto
def preprocess_text(text):
    stop_words = set(stopwords.words('spanish'))  # Eliminar palabras vacías en español
    lemmatizer = WordNetLemmatizer()
    
    # Tokenización
    tokens = word_tokenize(text.lower())
    # Filtrar palabras no deseadas
    filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    
    return " ".join(filtered_tokens)

preprocessed_documents = [preprocess_text(doc) for doc in documents]

# 3. Vectorización del texto
vectorizer = CountVectorizer()
document_term_matrix = vectorizer.fit_transform(preprocessed_documents)

# 4. Aplicar LDA con alpha y beta
# alpha (doc_topic_prior) controla la dispersión de los tópicos/temas en los documentos
# beta (topic_word_prior) controla la dispersión de las palabras en los tópicos/temas
lda = LatentDirichletAllocation(
    n_components=2,          # Número de tópicos/temas
    doc_topic_prior=0.1,     # Alpha: Mayor valor = documentos con múltiples tópicos/temas
    topic_word_prior=0.01,   # Beta: Mayor valor = temas con palabras más diversas
    random_state=42
)
lda.fit(document_term_matrix)

# 5. Mostrar los tópicos generados
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Tópico {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))
        print("\n")

num_top_words = 5
feature_names = vectorizer.get_feature_names_out()
display_topics(lda, feature_names, num_top_words)

# 6. Probabilidad de los tópicos en cada documento
topic_distribution = lda.transform(document_term_matrix)
topic_distribution = normalize(topic_distribution, norm='l1', axis=1)

for idx, probs in enumerate(topic_distribution):
    print(f"Documento {idx + 1}:")
    for topic_idx, prob in enumerate(probs):
        print(f"  - Tópico {topic_idx + 1}: {prob:.2f}")

Tópico 1:
aprendizaje ética fundamental desarrollo aplicaciones


Tópico 2:
artificial inteligencia industria computadora revolucionando


Documento 1:
  - Tópico 1: 0.02
  - Tópico 2: 0.98
Documento 2:
  - Tópico 1: 0.98
  - Tópico 2: 0.02
Documento 3:
  - Tópico 1: 0.98
  - Tópico 2: 0.02
Documento 4:
  - Tópico 1: 0.98
  - Tópico 2: 0.02
Documento 5:
  - Tópico 1: 0.02
  - Tópico 2: 0.98


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xiope\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xiope\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\xiope\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
